# Définition des prix des différents produits les plus smogglés

Commande XSV pour isoler les flows dans toflit de produits smogglés par les bateaux smogglers identifiés dans le fichier des bateaux smogglers 1787_10mois.

Nous avons ensuite analysés ces fichiers pour décider des prix les plus juste pour les produits smogglés.

Calculs des prix moyens/médians des produits ajoutés manuellement dans les fichiers _price.csv_, sauvegardés ensuite en _price_withaverage.csv_

In [52]:
# exports d'eaux-de-vie (catégorie large Révolution et Empire) vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "Eaux-de-vie et liqueurs" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/eauxdevie_price.csv

In [53]:
# exports de feuilles de tabac vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year \
| xsv search "feuilles de tabac" -s product_simplification\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/tabac_price.csv

In [54]:
# exports de thé vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "Thé" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/the_price.csv

In [55]:
# PAS UTILISÉE
# exports de soie vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "Soieries diverses" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/soieries_price.csv

Recherche des flux toflit18 "vins" afin de construire un prix "moyen" pour les eaux-de-vie, liqueurs ET vins, puis joindre nos analyses avec le module 4

In [56]:
# exports de vins
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "vins? " -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/vins_price.csv

Abandon de la piste de jointure des analyses avec le module 4 : le module 4 n'a pas de données pour 1787

In [57]:
# sucre brut
!xsv search Angleterre -s partner_grouping ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year| xsv search "sucre brut" -s product_simplification\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit\
> xsv_output_toflit18/sucre_price.csv

In [58]:
# café
!xsv search Angleterre -s partner_grouping ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year| xsv search "Café" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit\
> xsv_output_toflit18/cafe_price.csv

Les fichiers extraits par ces commandes sont sauvegardés à la racine du dossier

### Quantification du smogglage

Application des prix moyens/médians calculés depuis les données toflit18 (fichiers xsv_output_toflit18/*_price_withaverage.csv) sur les données du smogglage extraites de navigo (fichier dunkerque_smugglers_clean_20220126 - dunkerque_smugglers_clean_20220126.csv)

Le fichier principal du module n°5, _dunkerque_smugglers_clean_20220126 - dunkerque_smugglers_clean_20220126.csv_ ci-dessous, vient du drive, [à cette adresse](https://docs.google.com/spreadsheets/d/1oMqQpDWt77SVUqrkogA0GoMU4MzHla2GAuzp4bClii4/edit#gid=1993901892)

In [59]:
import csv

In [60]:
# load the data
smugglers = list(csv.DictReader(open("data/dunkerque_smugglers_clean_20220126 - dunkerque_smugglers_clean_20220126.csv", 'r')))

In [61]:
# calcul du prix d'une quantité de produit 
def get_priceandvalue(source_str_data, price, liters=False):
    if liters:
        quantity = int(source_str_data.replace("\xa0", "")) * 0.952
    else:
        quantity = int(source_str_data.replace("\xa0", ""))
    price = quantity * price
    return quantity, price

In [62]:
# variables declaration party

alcool_smuggle_quantity = 0
alcool_smuggle_price = 0

tobacco_smuggle_quantity = 0
tobacco_smuggle_price = 0

tea_smuggle_quantity = 0
tea_smuggle_price = 0

cafe_smuggle_quantity = 0
cafe_smuggle_price = 0

sucre_smuggle_quantity = 0
sucre_smuggle_price = 0

vins_smuggle_quantity = 0
vins_smuggle_price = 0

varia_smuggle_price = 0

In [63]:
# Do not forget to run the variable declaration party before re-running this
# Global price need to be put back to zero before re-run

for line in smugglers:

    # eaux de vie, geniève, tafia et liqueurs
    genieve = line["geniève (pintes de Paris)"]
    eauxdevie = line["eau-de-vie (pintes de Paris)"]
    tafia = line["Taffia (pintes de Paris)"]
    liqueurs = line["liqueurs (pintes)"]
    liters = True
    if genieve:
        genieve_quantity, genieve_price = get_priceandvalue(genieve, 0.62, liters)
        alcool_smuggle_quantity += genieve_quantity
        alcool_smuggle_price += genieve_price
    if eauxdevie:
        eauxdevie_quantity, eauxdevie_price = get_priceandvalue(eauxdevie, 0.62, liters)
        alcool_smuggle_quantity += eauxdevie_quantity
        alcool_smuggle_price += eauxdevie_price
    if tafia:
        tafia_quantity, tafia_price = get_priceandvalue(tafia, 0.62, liters)
        alcool_smuggle_quantity += tafia_quantity
        alcool_smuggle_price += tafia_price
    if liqueurs:
        liqueurs_quantity, liqueurs_price = get_priceandvalue(liqueurs, 0.62, liters)
        alcool_smuggle_quantity += liqueurs_quantity
        alcool_smuggle_price += liqueurs_price

    # feuilles de tabac
    tobacco = line["tabac en feuilles (livres poids)"]
    if tobacco:
        tobacco_quantity, tobacco_price = get_priceandvalue(tobacco, 0.36)
        tobacco_smuggle_quantity += tobacco_quantity
        tobacco_smuggle_price += tobacco_price

    # thés
    tea = line["thé (livres poids)"]
    if tea:
        tea_quantity, tea_price = get_priceandvalue(tea, 0.51)
        tea_smuggle_quantity += tea_quantity
        tea_smuggle_price += tea_price
    
    # café
    cafe = line["café (livres poids)"]
    if cafe:
        cafe_quantity, cafe_price = get_priceandvalue(cafe, 0.95)
        cafe_smuggle_quantity += cafe_quantity
        cafe_smuggle_price += cafe_price
        
    # sucre brut
    sucre = line["sucre brut (livres poids)"]
    if sucre:
        sucre_quantity, sucre_price = get_priceandvalue(sucre, 0.35)
        sucre_smuggle_quantity += sucre_quantity
        sucre_smuggle_price += sucre_price
    
    # vins
    vin_rouge = line["vin rouge (pintes)"]
    vin_fin = line["vin fin (pintes)"]
    liters = True
    # values not converted
    # barriques = line["vin (barriques)"]
    # pots = line["vin (pots)"]
    if vin_rouge:
        vin_rouge_quantity, vin_rouge_price = get_priceandvalue(vin_rouge, 1.8, liters)
        vins_smuggle_quantity += vin_rouge_quantity
        vins_smuggle_price += vin_rouge_price    
    if vin_fin:
        vin_fin_quantity, vin_fin_price = get_priceandvalue(vin_fin, 1.8, liters)
        vins_smuggle_quantity += vin_fin_quantity
        vins_smuggle_price += vin_fin_price
        
    # varia
    for product in line.keys():
        if "livres tournois" in product:
            if line[product]:
                varia_smuggle_price += int(line[product].replace("\xa0", ""))



La catégorie Varia contient les variables suivantes du fichier smugglers :

mousseline des Indes (livres tournois)
mousselines et mouchoirs (livres tournois)
mouchoirs de soie (livres tournois)
bandanoes (valeur livres tournois)
soie et mousseline (livres tournois)
marchandises des Indes (livres tournois)
soieries (livres tournois)
Cambray (en livres tournois)
nacres de perle (livres tournois)
porcelaine (livres tournois)

In [64]:
print("Export smugglés des eaux-de-vie et liqueurs en livres tournois : ", round(alcool_smuggle_price))
print("Export smugglés des eaux-de-vie et liqueurs en litres : ", round(alcool_smuggle_quantity))
print("\n")
print("Export smugglés de tabac en livres tournois : ", round(tobacco_smuggle_price))
print("Export smugglés de tabac en livres poids : ", tobacco_smuggle_quantity)
print("\n")
print("Export smugglés de thés en livres tournois : ", round(tea_smuggle_price))
print("Export smugglés de thés en livres poids : ", tea_smuggle_quantity)
print("\n")
print("Export smugglés de café en livres tournois : ", round(cafe_smuggle_price))
print("Export smugglés de café en livre poids : ", cafe_smuggle_quantity)
print("\n")
print("Export smugglés de sucre brut en livres tournois : ", round(sucre_smuggle_price))
print("Export smugglés de sucre brut en livre poids : ", sucre_smuggle_quantity)
print("\n")
print("Export smugglés de vins en livres tournois : ", round(vins_smuggle_price))
print("Export smugglés de vins en litres : ", round(vins_smuggle_quantity))
print("\n")
print("Export smugglés de catégorie varia en livres tournois : ", round(varia_smuggle_price))

Export smugglés des eaux-de-vie et liqueurs en livres tournois :  1069551
Export smugglés des eaux-de-vie et liqueurs en litres :  1725083


Export smugglés de tabac en livres tournois :  378815
Export smugglés de tabac en livres poids :  1052264


Export smugglés de thés en livres tournois :  17270
Export smugglés de thés en livres poids :  33863


Export smugglés de café en livres tournois :  17718
Export smugglés de café en livre poids :  18651


Export smugglés de sucre brut en livres tournois :  4138
Export smugglés de sucre brut en livre poids :  11824


Export smugglés de vins en livres tournois :  3379
Export smugglés de vins en litres :  1877


Export smugglés de catégorie varia en livres tournois :  143820


In [65]:
all_prices = sum([alcool_smuggle_price, tobacco_smuggle_price, tea_smuggle_price, cafe_smuggle_price, sucre_smuggle_price, vins_smuggle_price])
print("Export smogglés totaux : ", round(all_prices), " livres tournois")

Export smogglés totaux :  1490873  livres tournois


And now for something completely different

Tentative de calcul des valeurs par bateaux dans le fichier smugglers, afin de suivre les cargaisons (en valeur) jusqu'à leurs ports d'arrivée (ship_homeport dans les données smugglers)